### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. THis application will translate text from ENglish into another language. This is a relatively simple LLM application- it;s just a single  LLm call plus some prompting. Still, this is a  great wat to get started with Langchain - a lot of features can be built with just some prompting and an LLm call!

High level overview of :
- Using language models
- Using PrommptTempplates and OutputParsers
- Using LangChain Expression Language(LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying application with LangServe


In [11]:
### OpenAI API Key and open Source models -- Llama3, Gemma2, mistral -- Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


#### Groq
##### What is the LPU Inference Engine ?
An LPU Interface Engine, with LPU Standing for Language Processing Unit, is a hardware and software platform that delivers exceptional compute speed, quality and energy efficiency. This new type of end-to-end processing unit system provides the fastest inference for computationally intensive application with sequential componenets, such as AI lnaguage application like Large Lnaguage Models(LLMs).
##### Why is it so much faster than GPUs for LLMs and GenAI?
The LPU is designed to overcome the two LLM bottlenecks: Compute density and memory bandwidth. An LPU has greater compute capacity than a GPU and CPU in regards to LLMs. This reduces the amount of time per word calculated, allowing sequence to text to be generated much faster.

In [12]:
groq_api_key  = os.getenv("GROQ_KEY")
groq_api_key

'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'

In [9]:
!pip install langchain_openai


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [13]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [15]:
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EAD615AC90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EAD615A0C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))